# MPI and Elemental in Julia
## Parallel Workshop JuliaCon 2016
### `MPI.jl`
- MPI.jl provides
 - Julia wrappers for many MPI function but not yet all the newer ones. Normal script execution, e.g.
 ```
 mpirun -np 100000 julia mpiprogram.jl
 ```
 - An MPI Cluster manager for interactive execution of MPI jobs. See below

In [1]:
using MPI

Create an `MPIManager` and use `addprocs` to launch the workers. This will automatically initialize MPI.

In [2]:
man = MPIManager(np = 8)
addprocs(man)

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

To run a command on the MPI workers, use the `@mpi_do` macro. Here, store the MPI rank in a variable and `@show` it

In [3]:
@mpi_do man @show myrank = MPI.Comm_rank(MPI.COMM_WORLD);

	From worker 2:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 0
	From worker 9:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 7
	From worker 3:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 1
	From worker 7:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 5
	From worker 5:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 3
	From worker 8:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 6
	From worker 4:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 2
	From worker 6:	myrank = MPI.Comm_rank(MPI.COMM_WORLD) = 4


Allocate the vector `x` on all workers and show its values. Notice that the RNG is not syncronized accross workers.

In [4]:
@mpi_do man @show x = randn(2)

	From worker 3:	x = randn(2) = [1.561807228850006,-1.8991612031066882]
	From worker 4:	x = randn(2) = [1.7827074625350516,1.1432842639087033]
	From worker 5:	x = randn(2) = [0.7812067994558566,1.3515494345331944]
	From worker 9:	x = randn(2) = [-1.0100482065707683,0.46991613460582093]
	From worker 7:	x = randn(2) = [-0.4051027120044051,-0.8893470928293499]
	From worker 6:	x = randn(2) = [-0.7893776602217285,-2.3168249675165273]
	From worker 2:	x = randn(2) = [-2.7318205254023775,0.6965055849677959]
	From worker 8:	x = randn(2) = [0.4709601667537731,-1.1963960727081422]


Send and receive.

In [7]:
@mpi_do man begin
    if myrank == 3
        MPI.Send(3.0, 2, 0, MPI.COMM_WORLD)
        elseif myrank == 2
        MPI.Recv!(x, 1, 3, 0, MPI.COMM_WORLD)
    end
end
@mpi_do man @show x

	From worker 7:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 4:	x = [3.0,0.6965055849677959]
	From worker 3:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 2:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 5:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 8:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 6:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 9:	x = [-2.7318205254023775,0.6965055849677959]


Below, we show an example of `Bcast!` which is the Julia wrapper function for the collective MPI operation `MPI_Bcast`. The function is overloaded for several input types. The type of the broadcasted buffer is always determined from the Julia type and if the input argument is a Julia vector then the size determined automatically as well. Alternatively if the argument is either a vector or pointer, the length can be specified as an integer argument.

In [8]:
@mpi_do man MPI.Bcast!(x, 0, MPI.COMM_WORLD)
@mpi_do man @show x

	From worker 7:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 4:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 9:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 2:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 3:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 5:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 8:	x = [-2.7318205254023775,0.6965055849677959]
	From worker 6:	x = [-2.7318205254023775,0.6965055849677959]


Compute $\pi$ with `MPI_Reduce` (`MPI_Allreduce` available in `MPI.jl` development version)

In [29]:
@mpi_do man begin
    n = 10^7
    
    # compute pi locally
    pi_local = mapreduce(i -> rand()^2 + rand()^2 < 1, +, 1:n)/n*4
    
    # combine the results with MPI_Reduce
    pi_final = MPI.Reduce(pi_local, MPI.SUM, 0, MPI.COMM_WORLD)

    # show the result
    if myrank == 0
        @show pi_final / MPI.Comm_size(MPI.COMM_WORLD)
    end
end

	From worker 2:	pi_final / MPI.Comm_size(MPI.COMM_WORLD) = 3.1414827


## `Elemental.jl`

- [Elemental](http://github.com/Elemental/elemental) is a C++ library for distributed dense linear algebra (lately also some sparse and optimization functions)
- `Elemental.jl` provides Julia wrappers for `Elemental`.
- Still alpha stage
- Two APIs
 - Thin layer on top of C++ library
 - Higher level with `DArray` interoperability

In [30]:
using Elemental

In [31]:
# Define a 2000x2000 Elemental distributed matrix of Gaussian variates
@mpi_do man n = 2000
@mpi_do man A = Elemental.DistMatrix(Float64)
@mpi_do man Elemental.gaussian!(A, n, n)

In [32]:
# Print the first element
@mpi_do man @show A[1,1]

	From worker 2:	A[1,1] = -1.0290752082028511
	From worker 4:	A[1,1] = -1.0290752082028511
	From worker 3:	A[1,1] = -1.0290752082028511
	From worker 6:	A[1,1] = -1.0290752082028511
	From worker 5:	A[1,1] = -1.0290752082028511
	From worker 9:	A[1,1] = -1.0290752082028511
	From worker 7:	A[1,1] = -1.0290752082028511
	From worker 8:	A[1,1] = -1.0290752082028511


In [33]:
# Compute the singular values with Elementals singular value solver. Computes all values.
@time @mpi_do man vals = svdvals(A)

  6.251383 seconds (5.46 k allocations: 413.798 KB)


In [34]:
# Show the largest singular value
@mpi_do man @show vals[1]

	From worker 6:	vals[1] = 89.16743025999959
	From worker 4:	vals[1] = 89.16743025999959
	From worker 3:	vals[1] = 89.16743025999959
	From worker 2:	vals[1] = 89.16743025999959
	From worker 7:	vals[1] = 89.16743025999959
	From worker 5:	vals[1] = 89.16743025999959
	From worker 8:	vals[1] = 89.16743025999959
	From worker 9:	vals[1] = 89.16743025999959


In [35]:
# Now compate to a local compuation with Julia's `svdvals` based on LAPACK
n = 2000
A = randn(n, n);
@time @show svdvals(A)[1];

(svdvals(A))[1] = 88.76880541870597
  5.691730 seconds (90.51 k allocations: 35.761 MB, 0.06% gc time)


In [38]:
# Now try out my TSVD package on an Elemental array

# Pkg.clone("https://github.com/andreasnoack/TSVD.jl")
using TSVD

In [41]:
# PROBLEM: The tsvd function uses random seed on all workers which we saw was different for each worker.
# SOLUTION: Use Bcast! (or just set the seed on all workers)
@time @mpi_do man begin
    v0 = randn(n)
    MPI.Bcast!(v0, 0, MPI.COMM_WORLD)
    
    # compute the SVD
    vals = TSVD.tsvd(A, 10, initVec = v0)
end

  1.757477 seconds (5.88 k allocations: 444.171 KB)
